In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

# Read Holiday-CSV from Kaggle

In [47]:
df = pd.read_csv('holiday_kaggle.csv', sep=';')

In [48]:
df

,date,holiday,country,iso2,country_official_name
0,2020-02-24,Día de Carnaval [Carnival's Day],Argentina,AR,Argentine Republic
1,2020-02-25,Día de Carnaval [Carnival's Day],Argentina,AR,Argentine Republic
2,2020-03-24,Día Nacional de la Memoria por la Verdad y la ...,Argentina,AR,Argentine Republic
3,2020-04-09,Semana Santa (Jueves Santo) [Holy day (Holy T...,Argentina,AR,Argentine Republic
4,2020-04-10,Semana Santa (Viernes Santo) [Holy day (Holy ...,Argentina,AR,Argentine Republic
...,...,...,...,...,...
1718,2021-05-09,Victory Day and Commemoration of the Fallen He...,Moldova,MD,Republic of Moldova
1719,2021-06-01,Children's Day,Moldova,MD,Republic of Moldova
1720,2021-08-27,Independence Day,Moldova,MD,Republic of Moldova
1721,2021-08-31,National Language Day,Moldova,MD,Republic of Moldova


# Scrape ISO-3 from IBAN website

In [49]:
url = 'https://www.iban.com/country-codes'

r = requests.get(url)
html = r.text
soup = BeautifulSoup(html)
table = soup.find('table', {"class": "table table-bordered downloads tablesorter"})
rows = table.find_all('tr')
data = []
for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])
    
df_iso = pd.DataFrame(data, columns=['country', 'iso-2', 'iso-3', 'code'])

In [50]:
df_iso

,country,iso-2,iso-3,code
0,Afghanistan,AF,AFG,004
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,008
3,Algeria,DZ,DZA,012
4,American Samoa,AS,ASM,016
...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876
245,Western Sahara,EH,ESH,732
246,Yemen,YE,YEM,887
247,Zambia,ZM,ZMB,894


# Join iban.ISO-3 on holidays.Country

In [251]:
df_summary = pd.merge(df_summary, df_iso, on=['country', 'country'])
df_summary.drop(['iso-2','code'], axis=1, inplace=True)

In [252]:
df_summary

,2016,2017,2018,2019,2020,2021,2022,country,iso-3


In [59]:
df_holiday.to_csv('holiday_processed.csv', index=False)

# Test for Flask

In [12]:
temp = df_scraped.loc[(df_scraped['country'] == 'Sweden') & (df_scraped['date'] == '2021-12-24')].values[0][1]

In [7]:
df_scraped

,date,holiday,country,iso
0,2018-01-29,Public Holiday,Afghanistan,AFG
1,2018-02-15,Liberation Day,Afghanistan,AFG
2,2018-02-28,Public Holiday,Afghanistan,AFG
3,2018-03-21,Nowrooz,Afghanistan,AFG
4,2018-03-22,Nowrooz Holiday,Afghanistan,AFG
...,...,...,...,...
20051,2022-08-08,Heroes' Day,Zimbabwe,ZWE
20052,2022-08-09,Defence Forces Day,Zimbabwe,ZWE
20053,2022-12-22,Unity Day,Zimbabwe,ZWE
20054,2022-12-25,Christmas Day,Zimbabwe,ZWE


In [4]:
df_scraped = pd.read_csv('ibm_docker/holiday_scraped.csv')

In [266]:
df_summary = df_scraped.groupby(['country', 'iso']).count()
    
for year in df_scraped.date.astype(str).str[:4].unique():
    df_tmp = df_scraped[df_scraped.date.astype(str).str[:4] == year]
    df_tmp = df_tmp.groupby(['country', 'iso']).count()
    df_summary[year] = df_tmp['date']
    
df_summary.reset_index(inplace=True)
df_summary.drop(['holiday', 'date'], axis=1, inplace=True)
df_summary = df_summary.reindex(sorted(df_summary.columns), axis=1)

In [267]:
df_summary

,2016,2017,2018,2019,2020,2021,2022,country,iso
0,NaN,NaN,18.0,22.0,18,17,17,Afghanistan,AFG
1,16.0,14.0,14.0,15.0,16,15,18,Albania,ALB
2,9.0,9.0,10.0,11.0,11,11,11,Algeria,DZA
3,NaN,NaN,NaN,14.0,15,18,17,American Samoa,ASM
4,NaN,NaN,14.0,14.0,14,14,14,Andorra,AND
...,...,...,...,...,...,...,...,...,...
173,NaN,NaN,NaN,14.0,19,15,16,Vanuatu,VUT
174,NaN,NaN,NaN,NaN,13,13,13,Wallis and Futuna,WLF
175,18.0,16.0,18.0,17.0,17,17,17,Yemen,YEM
176,17.0,15.0,17.0,15.0,14,15,16,Zambia,ZMB


In [213]:
df_summary1

,country,iso-3,2020,2021
0,Albania,ALB,13,15
1,Argentina,ARG,17,18
2,Australia,AUS,8,9
3,Austria,AUT,11,13
4,Bangladesh,BGD,7,7
...,...,...,...,...
57,Switzerland,CHE,8,9
58,Thailand,THA,12,12
59,Turkey,TUR,7,8
60,Ukraine,UKR,9,11


In [312]:
r = requests.get('http://localhost:5000/api?country=Sweden&date=2022-12-24')
json = r.json()

In [313]:
json

False

In [11]:
df_scraped.iso.unique()

array(['AFG', 'ALB', 'DZA', 'ASM', 'AND', 'AGO', 'AIA', 'ATG', 'ARG',
       'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHR', 'BGD', 'BRB', 'BLR',
       'BEL', 'BLZ', 'BEN', 'BMU', 'BTN', 'BES', 'BIH', 'BWA', 'BRA',
       'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'TCD', 'CHL', 'CHN',
       'COL', 'CRI', 'HRV', 'CUB', 'CUW', 'CYP', 'DNK', 'DJI', 'DMA',
       'ECU', 'EGY', 'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'FJI',
       'FIN', 'FRA', 'GUF', 'PYF', 'GAB', 'GEO', 'DEU', 'GHA', 'GIB',
       'GRC', 'GRL', 'GRD', 'GLP', 'GUM', 'GTM', 'GGY', 'GIN', 'GNB',
       'GUY', 'HTI', 'HND', 'HKG', 'HUN', 'ISL', 'IND', 'IDN', 'IRQ',
       'IRL', 'IMN', 'ISR', 'ITA', 'JAM', 'JPN', 'JEY', 'JOR', 'KAZ',
       'KEN', 'KIR', 'KWT', 'KGZ', 'LVA', 'LBN', 'LSO', 'LBR', 'LBY',
       'LIE', 'LTU', 'LUX', 'MDG', 'MWI', 'MYS', 'MDV', 'MLI', 'MLT',
       'MTQ', 'MRT', 'MUS', 'MEX', 'MCO', 'MNG', 'MNE', 'MSR', 'MAR',
       'MOZ', 'MMR', 'NAM', 'NPL', 'NCL', 'NZL', 'NIC', 'NGA', 'NOR',
       'OMN', 'PAK',